In [102]:
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

In [103]:
sizeOfSets = 500
sizeOfTweetContent = 75
val = [None] * 4000
document = [""] * sizeOfSets

In [104]:
categories = ['airlines', 'celebrities', 'colleges', 'fastfood', 'leagues', 'news', 'streamingplatforms', 'techgiants']

In [105]:
overCount= 0
for i in categories:
    direc:str = '/Users/sameernawaz/AnacondaProjects/Datasets/tweets/'+i+'/trainingTextClassification.csv'
    with open(direc, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        count = 0
        for line in csv_reader:
            if len(line[6]) > sizeOfTweetContent:
                val[overCount] = line
                count += 1
                overCount += 1
                if count >= sizeOfSets :
                    break

In [106]:
print(val[0])

['2019-11-28 15:39:14', 'Ryanair', 'hanruth19', '0', '0', '0', 'Hi Hannah, I hope your father will get well soon. Please contact me via DM, so I can look into it. Balazs', '', '', '', '1200076692229898245', 'https://twitter.com/Ryanair/status/1200076692229898245']


In [107]:
df = pd.DataFrame(val, columns = ['date', 'username', 'to', 'replies', 
                                  'retweets', 'favorites', 'text', 'geo', 
                                  'mentions', 'hashtags', 'id', 'permalink']) 

In [110]:
df = df.assign(rec_id=np.arange(len(df))).reset_index(drop=True)

In [113]:
df = df[['rec_id', 'username', 'text']]
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 0 to 3999
Data columns (total 3 columns):
rec_id      4000 non-null int64
username    4000 non-null object
text        4000 non-null object
dtypes: int64(1), object(2)
memory usage: 125.0+ KB


In [114]:
import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df['text']))
len(norm_corpus)

4000

In [115]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(4000, 10013)

In [116]:
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,3990,3991,3992,3993,3994,3995,3996,3997,3998,3999
0,1.000000,0.029019,0.038162,0.000000,0.070939,0.340215,0.000000,0.008397,0.055904,0.082092,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.029019,1.000000,0.201166,0.278895,0.030014,0.064866,0.087389,0.009486,0.025585,0.056975,...,0.0,0.0,0.0,0.04082,0.0,0.0,0.0,0.0,0.0,0.0
2,0.038162,0.201166,1.000000,0.034706,0.057251,0.043692,0.000000,0.000000,0.051894,0.024882,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.278895,0.034706,1.000000,0.074939,0.000000,0.159287,0.026320,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.070939,0.030014,0.057251,0.074939,1.000000,0.081218,0.091611,0.008539,0.099868,0.083479,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0


In [118]:
movies_list = df['rec_id'].values
movies_list, movies_list.shape

(array([   0,    1,    2, ..., 3997, 3998, 3999]), (4000,))

In [122]:
movie_idx = np.where(movies_list == 3997)[0][0]
movie_idx

3997

In [123]:
movie_similarities = doc_sim_df.iloc[movie_idx].values
movie_similarities

array([0., 0., 0., ..., 1., 0., 0.])

In [124]:
similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
similar_movie_idxs

array([3961, 3989, 3957, 3978, 3986])

In [125]:
similar_movies = movies_list[similar_movie_idxs]
similar_movies

array([3961, 3989, 3957, 3978, 3986])

In [126]:
def movie_recommender(movie_title, movies=movies_list, doc_sims=doc_sim_df):
    # find movie id
    movie_idx = np.where(movies == movie_title)[0][0]
    # get movie similarities
    movie_similarities = doc_sims.iloc[movie_idx].values
    # get top 5 similar movie IDs
    similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
    # get top 5 movies
    similar_movies = movies[similar_movie_idxs]
    # return the top 5 movies
    return similar_movies

In [131]:
post_id = 3957

print((df.loc[df['rec_id'] == post_id]['text']).values[0])
print('\n')
for i in movie_recommender(movie_title = post_id, movies=movies_list, doc_sims=doc_sim_df):
    print("=>",(df.loc[df['rec_id'] == i]['text']).values[0])

RT @mangrovemike Just installed my 14th software update on my #Tesla P85 since Dec 2014. New UI and features. I love this car @TeslaMotors


=> Wish granted . New Sketchpad features are rolling out in our next software update. What will you draw? 
=> Welcome to the family! RT @KP24 Just been designing my new @TeslaMotors! What a car, what a company! https://twitter.com/teslamotors/status/742746893101367296 …
=> TeslAtari updates inbound! Look for Super Breakout and more in the latest software update which will begin rolling out later this week 
=> It's in our V .̷̛͙̪̀͑̐͌̍͛̑͆͗͐̀͆̿͗͘̕͠͠.̶̨̢̧̧̛̫̱̯̠̮͇̤̠̫̯̺̪̭̻͇͚̅̎͂̀̀̔̃̃́̈́̈́̒͋̏̓̓̓̓̀̆̄͋̎̈̏͠ͅ update
=> Keeping it simple RT @Jon4Lakers I just ordered a car online. We live in the future @TeslaMotors
